In [1]:
# NOTE: export SPARK_DRIVER_MEMORY=1g

In [2]:
import keras
from keras.applications import ResNet50
from keras.applications import imagenet_utils

Using TensorFlow backend.


* For resnet, we only support NCHW and Keras 1.2.2 for now

In [3]:
keras.backend.set_image_dim_ordering("th")

* Create ResNet50 using Keras

In [4]:
keras_model = ResNet50(weights="imagenet")

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python2.7/dist-packages/keras/applications/resnet50.py:231: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


* Init BigDL and spark environment

In [5]:
from bigdl.keras.backend import *
from bigdl.transform.vision.image import *

redire_spark_logs()
show_bigdl_info_logs()
init_engine()
sparkConf = create_spark_conf().setAppName("test model")
sc = get_spark_context(sparkConf)

* Load a Keras definition and weights to BigDL.

In [6]:
bmodel = DefinitionLoader.from_kmodel(keras_model)
WeightLoader.load_weights_from_kmodel(bmodel, keras_model)

creating: createInput
creating: createSequential
creating: createPadding
creating: createPadding
creating: createPadding
creating: createPadding
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization


/tmp/spark-c4146c33-fb9b-465e-8428-944eecbabf99/userFiles-b51ed05e-a098-4c0d-bf7b-e900b11738ed/analytics-zoo-bigdl_0.6.0-spark_2.3.1-0.3.0-python-api.zip/bigdl/keras/converter.py:791: UserWarning: Cannot find dim_ordering from json definition. Using the default instead.


creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createSpatialBatchNormalization
creating: createCAddTable
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createCAddTable
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: 

* Load data into ImageFrame

In [7]:
image_frame = ImageFrame.read("image-net", sc=sc)

* Define prepocess transformer

In [8]:
transformer = Pipeline([BytesToMat(), Resize(256, 256),
                        CenterCrop(224, 224),
                        ChannelNormalize(123.68, 103.939, 116.779),
                        MatToTensor(), ImageFrameToSample()])

creating: createBytesToMat
creating: createResize
creating: createCenterCrop
creating: createChannelNormalize
creating: createMatToTensor
creating: createImageFrameToSample
creating: createPipeline


* Run prediction

In [9]:
preds_frame = bmodel.predict_image(image_frame.transform(transformer))

* Show result of the first image

In [ ]:
preds = preds_frame.get_predict().take(1)

for pre in preds:
    path, pre_score = pre
    P = imagenet_utils.decode_predictions(np.expand_dims(pre_score, axis=0))
    print("Prediction for {}".format(path))
    # Display the top-5 prediction
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("    {}. {}: {:.2f}%".format(i + 1, label, prob * 100))

* While the origin image is
![scalars](image-net/158516268_796c3e047d.jpg)